<a href="https://colab.research.google.com/github/supplient/bachelor_design/blob/equal_realize/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
!git clone https://github.com/supplient/bachelor_design.git
import os
os.chdir("bachelor_design")
!pwd

Cloning into 'bachelor_design'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 316 (delta 43), reused 57 (delta 32), pack-reused 243
Receiving objects: 100% (316/316), 5.97 MiB | 4.10 MiB/s, done.
Resolving deltas: 100% (174/174), done.
/content/bachelor_design


In [0]:
!git checkout equal_realize
!git pull

Branch 'equal_realize' set up to track remote branch 'equal_realize' from 'origin'.
Switched to a new branch 'equal_realize'
Already up to date.


# Mount GDriver
Perpare Colab environment.

In [0]:
from driver_amount import addh

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
[Colab] Using address head: /gdrive/My Drive


In [0]:
!pip install keras_bert

  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=453ef8666f0c2ba38a34fc10b8a929cdab1503952d30fd9df99d01d63db17b29
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-transformer: filename=keras_transformer-0.32.0-cp36-none-any.whl size=13266 sha256=92dcb4ceb9db767edc0eb6bcad18960a77285146f4d85c57e40702bea57a086d
  Stored in directory: /root/.cache/pip/wheels/62/f0/ce/82fa5d024d5ef8e263f26a50dcee23820efe245680ce9c922a
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=4e2bd650532558fe5f07fa42e99f5d730f79196e1d96a70d72413b4dde7f4142
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=a068b390a16067b1cc5c22c0ecc4acece0e4a80e2a7e5f53a6d97051f37e56bb
  Stor

In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-dpbozcvl
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-dpbozcvl
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=e8b0d408504e454a7335a4e907fc1c53422011c15fc2d00d255db729e34292c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-dnttqkqd/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
import os
os.chdir("/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/")
lines = []
with open("crf.py", "r") as fd:
  lines = fd.readlines()
lines[515] = "            mask2 = K.cast(K.concatenate([mask, K.cast(K.zeros_like(mask[:, :1]), mask.dtype)], axis=1),\n"
with open("crf.py", "w") as fd:
  fd.writelines(lines)

os.chdir("/content")

In [0]:
os.chdir("bachelor_design")

# Prepare Input

In [0]:
from cut_and_tag import cut_and_tag
import preprocess
import config

In [0]:
cut_seqs, char_seqs, tag_seqs = cut_and_tag(
    addh + config.DATA_PATH,
    addh + config.STOPWORDS_PATH
)
char_seqs, tag_seqs = preprocess.shuffle_twin(
    char_seqs,
    tag_seqs
)
token_id_seqs, segment_seqs, tag_id_seqs, tag_vocab = preprocess.preprocess(
    char_seqs, 
    tag_seqs,
    addh + config.BERT_VOCAB_PATH,
    SEQ_LEN=config.SEQ_LEN
    )

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.907 seconds.
Prefix dict has been built successfully.
Using TensorFlow backend.


In [0]:
# Split train and text
train_num = int(len(token_id_seqs) * 0.9)

test_token_id_seqs = token_id_seqs[train_num:]
test_segment_seqs = segment_seqs[train_num:]
test_tag_id_seqs = tag_id_seqs[train_num:]

train_token_id_seqs = token_id_seqs[0:train_num]
train_segment_seqs = segment_seqs[0:train_num]
train_tag_id_seqs = tag_id_seqs[0:train_num]

In [0]:
import json
with open(addh + config.TAG_VOCAB_PATH, "w") as fd:
    json.dump(tag_vocab, fd)

In [0]:
tag_vocab

{'': 0,
 'B-EI': 6,
 'B-EIF': 10,
 'B-EO': 2,
 'B-EQ': 8,
 'B-ILF': 4,
 'I-EI': 7,
 'I-EIF': 11,
 'I-EO': 3,
 'I-EQ': 9,
 'I-ILF': 5,
 'O': 1}

# Build Model
using BERT

In [0]:
import keras
import keras_bert 

In [0]:
bert_model = keras_bert.load_trained_model_from_checkpoint(
    # config_file
    addh + config.BERT_CONFIG_PATH, 
    # checkpoint file
    addh + config.BERT_CHECKPOINT_PATH, 
    # settings
    training=False, # Not train the whole model. Ignore NSP and MLM
    trainable=True

)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









In [0]:
from keras_contrib.layers import CRF

In [0]:
input_token = keras.layers.Input(shape=(config.SEQ_LEN,))
input_segment = keras.layers.Input(shape=(config.SEQ_LEN,))

bert_output = bert_model([input_token, input_segment])

crf_model = CRF(len(tag_vocab), sparse_target=True)

output = crf_model(bert_output)

model = keras.models.Model([input_token, input_segment], output)

In [0]:
model.compute_output_shape([(1, config.SEQ_LEN), (1, config.SEQ_LEN)])

(1, 512, 12)

In [0]:
bert_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [0]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, 512, 768)     177262848   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
crf_1 (CRF)                     (None, 512, 12)      9396        model_2[1][0]              

# Fine-tuning

In [0]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00004),
    loss=crf_model.loss_function,
    metrics=[crf_model.accuracy]
)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [0]:
import epoch_checkpoint
epoch_callback = epoch_checkpoint.EpochCheckpoint(
    addh + config.MODEL_PATH,
    addh + config.TRAIN_REC_PATH,
    1,
    [test_token_id_seqs, test_segment_seqs],
    test_tag_id_seqs,
    tag_vocab
    )

In [0]:
model.fit(
    x=[train_token_id_seqs, train_segment_seqs],
    y=[train_tag_id_seqs], 
    batch_size=8, 
    epochs=15,
    verbose=1,
    validation_data=[[test_token_id_seqs, test_segment_seqs], [test_tag_id_seqs]],
    callbacks=[epoch_callback]
)



Train on 7005 samples, validate on 779 samples
Epoch 1/15
7005/7005 [==============================] - 2639s 377ms/step - loss: 55.9537 - crf_viterbi_accuracy: 0.8192 - val_loss: 57.2379 - val_crf_viterbi_accuracy: 0.8276
Saving checkpint...
Predicing for matrics calculating...
779/779 [==============================] - 121s 155ms/step
Updating train record...
Epoch 2/15
7005/7005 [==============================] - 2655s 379ms/step - loss: 55.7731 - crf_viterbi_accuracy: 0.8597 - val_loss: 57.1286 - val_crf_viterbi_accuracy: 0.8475
Saving checkpint...
Predicing for matrics calculating...
779/779 [==============================] - 120s 153ms/step
Updating train record...
Epoch 3/15
7005/7005 [==============================] - 2652s 379ms/step - loss: 55.6656 - crf_viterbi_accuracy: 0.8864 - val_loss: 57.1424 - val_crf_viterbi_accuracy: 0.8573
Saving checkpint...
Predicing for matrics calculating...
779/779 [==============================] - 120s 153ms/step
Updating train record...
Epo

In [0]:
model.save(addh + config.MODEL_PATH)